In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize



Autosaving every 180 seconds


<ipython-input-1-eeb477d8c4a5>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
#################################################
########### VISUALIZE RAW PIXEL STMS ############
#################################################

fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr1_30Hz/IJ1pm_Apr1_30Hz_12.0sec_chebyshev_globalAverage_0.1hz_6.0hz_04code_stm_mean.npy'
data = np.float32(np.load(fname))
print (data.shape)

plt.figure()
hstack = []
window = 10
n_sec = 5                     # 5 seconds out
start = (12-n_sec)*30
end = data.shape[0]//2
for k in range(start,end,window):
    temp = data[k:k+window].mean(0)
    hstack.append(temp)
    
hstack=np.hstack(hstack)    
vmin = np.min(hstack)
vmax = np.max(hstack)
plt.imshow(hstack,
           vmin=vmin,
           vmax=vmax
          #aspect='auto'
          )
plt.show()

(721, 128, 128)


In [19]:
#################################################
#################################################
#################################################
def butter_lowpass(cutoff, nyq_freq, order=4):
    from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y

    
animal_id = 'AQ2'
lockout = 15
animal_ids = ["IA1", "IA2","IA3","IJ1","IJ2","AQ2"]
#animal_ids = ['IA1']
ctr=0
plt.figure(figsize=(15,10))
for animal_id in animal_ids: 
    ax=plt.subplot(2,3,ctr+1)
    #
    if animal_id == "AQ2":

        fname1 = '/media/cat/4TBSSD/yuki/'+animal_id+'/tif_files/'+animal_id+'pm_Mar7_Day3_30Hz/'+animal_id+'pm_Mar7_Day3_30Hz_locanmf_wholestack.npz'

        fname2 = '/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz'
    else:
        fname1 = '/media/cat/4TBSSD/yuki/'+animal_id+'/tif_files/'+animal_id+'am_Mar7_30Hz/'+animal_id+'am_Mar7_30Hz_locanmf_wholestack.npz'

        fname2 = '/media/cat/4TBSSD/yuki/'+animal_id+'/super_sessions/alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz'

    data1 = np.load(fname1,allow_pickle=True)
    names = data1['names']
    #print (names)

    # 
    data2 = np.load(fname2, allow_pickle=True)

    trials = np.vstack(data2['trials'])
    print (trials.shape)

    #
    #means = trials.mean(0)
    if False:
        from tqdm import trange
        trials_norm = np.zeros(trials.shape)
        for k in trange(trials.shape[0]):
            max_ = np.max(trials[k])
            for p in range(trials.shape[1]):
                #trials_norm[k,p] = trials[k,p]/np.max(trials[k,p])  # normalize by the individual areas
                trials_norm[k,p] = trials[k,p]/max_  # normalize by the highest limb area
        print (trials_norm.shape)
        trials = trials_norm
        
    means = np.median(trials,axis=0)
    
    if True:
        max_ = np.max(means)
        for k in range(means.shape[1]):
            means[:,k] = means[:,k]/max_
            
    # buttefilter
    if False:
        cutoff_freq = .5
        nyq_freq = 30//2
        for k in range(means.shape[1]):
            means[:,k] = butter_lowpass_filter(means[:,k], cutoff_freq, nyq_freq, order=4)

    # 
    t = np.arange(means.shape[0])/30-30
    #for k in range(means.shape[1]):
    for k in range(8,16,1):
        plt.plot(t,means[:,k], label=names[k],
                 linewidth=3,
                 alpha=.8)
    if ctr==0:
        plt.legend(fontsize=7)
        
    plt.plot([-30,30],[0,0],'--',c='black',
             linewidth=5)
    plt.title(animal_id)
    plt.xticks([])
    plt.ylim(-1,1)
    #plt.yticks([])
    ctr+=1
    plt.xlim(-10,0)
plt.suptitle(animal_id+"  lockout: "+str(lockout))
if True:
    plt.savefig('/home/cat/ROI_averages_musall_'+str(lockout)+'.svg')
    plt.close()
else:
    plt.show()


(503, 1800, 16)
(564, 1800, 16)
(188, 1800, 16)
(261, 1800, 16)
(139, 1800, 16)
(594, 1800, 16)


['Retrosplenial area, dorsal part, layer 1 - right'
 'Retrosplenial area, lateral agranular part, layer 1 - right'
 'Primary visual area, layer 1 - right'
 'Primary somatosensory area, upper limb, layer 1 - right'
 'Primary somatosensory area, lower limb, layer 1 - right'
 'Primary somatosensory area, barrel field, layer 1 - right'
 'Secondary motor area, layer 1 - right'
 'Primary motor area, Layer 1 - right'
 'Primary motor area, Layer 1 - left'
 'Secondary motor area, layer 1 - left'
 'Primary somatosensory area, barrel field, layer 1 - left'
 'Primary somatosensory area, lower limb, layer 1 - left'
 'Primary somatosensory area, upper limb, layer 1 - left'
 'Primary visual area, layer 1 - left'
 'Retrosplenial area, lateral agranular part, layer 1 - left'
 'Retrosplenial area, dorsal part, layer 1 - left']


In [ ]:
###########################################
############ LOAD LOCANMF COMPS ###########
###########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
#session_id = 'IA1pm_Feb3_30Hz'

# animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'


sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

# look at specific trials and neural activity
fname = os.path.join(
    root_dir, 
    animal_id,
    'tif_files',
    session_id,
    session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

data = np.load(fname, allow_pickle=True)
names_behaviors = data['names']
print ("names: ", names_behaviors)
trials = data['trials']
print (trials.shape)
random = data['random']
behavior_times = data['behaviors']
print (behavior_times.shape)
# t = np.arange(trials.shape[1])/30.-30
# plt.plot(t,trials[2,:,:])
# plt.show()

shift = np.loadtxt(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'shift.txt'))
                   
print ("Shifts: ", shift)

blue_light = np.load(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'blue_light_frame_triggers.npz'))

start_blue = blue_light['start_blue']/30
print ("start blue: ", start_blue)

fname = os.path.join(
                root_dir, 
                animal_id,
                'tif_files',
                session_id,
                session_id+'_locanmf_wholestack.npz')

data = np.load(fname, allow_pickle=True)

names = data['areanames_area']
areas = data['areas']
names2 = data['names']
A = data["A_reshape"]
print (A.shape)
whole_stack = data["whole_stack"]
print (whole_stack.shape)

clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
ctr_clrs =0
#
for k in range(len(names2)):
    print (k, names2[k])

area_ids = [12, 8, 13]  # left limb, left M1, left visual
ca = whole_stack[area_ids]
print (ca.shape)

#
plt.figure()
t = np.arange(ca.shape[1])/30.

# plot calcium activity
for k in range(ca.shape[0]):
    temp = ca[k]
    temp = temp - np.median(temp) + k*350
    plt.plot(t,temp, 
             c=clrs[ctr_clrs],
             linewidth=2,
             label = names2[area_ids][k])

    ctr_clrs+=1
    
# plot rewarded lever pulls
temp = behavior_times[0]/30.
plt.axvline(x=temp[0]
            #+shift[0]
            +start_blue
            ,linestyle='--',
            c=clrs[ctr_clrs],
           label='rewarded lever pull')
for k in range(1,temp.shape[0],):
    plt.axvline(x=temp[k]
                #+shift[0]
                +start_blue
            ,linestyle='--',

                c=clrs[ctr_clrs])
ctr_clrs+=1
    
# plot rest of events
ctr=0
for k in [2,3,7]:
    temp = behavior_times[k]
    print (temp.shape)
    temp = temp/30+ start_blue
    plt.scatter(temp , 
                temp*0+1300+ ctr*50, 
                c=clrs[ctr_clrs],
                label=names_behaviors[k])
    ctr+=1
    ctr_clrs+=1
    
plt.legend()
plt.show()



In [2]:
##################################################
########## MAKE BEHAVIOR SEQUENCES VECTORS #######
##################################################

def get_lever_pulls_and_behaviors(session_id, root_dir, animal_id):

    # look at specific trials and neural activity
    fname = os.path.join(
        root_dir, 
        animal_id,
        'tif_files',
        session_id,
        session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    names_behaviors = data['names']
    print ("names: ", names_behaviors)
    trials = data['trials']
    print (trials.shape)
    random = data['random']
    behavior_times = data['behaviors']
    print (behavior_times.shape)
    # t = np.arange(trials.shape[1])/30.-30
    # plt.plot(t,trials[2,:,:])
    # plt.show()

    shift = np.loadtxt(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'shift.txt'))

    print ("Shifts: ", shift)

    blue_light = np.load(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'blue_light_frame_triggers.npz'))

    start_blue = blue_light['start_blue']/30
    print ("start blue: ", start_blue)

    fname = os.path.join(
                    root_dir, 
                    animal_id,
                    'tif_files',
                    session_id,
                    session_id+'_locanmf_wholestack.npz')

    data = np.load(fname, allow_pickle=True)

    names = data['areanames_area']
    areas = data['areas']
    names2 = data['names']
    A = data["A_reshape"]
    print (A.shape)
    whole_stack = data["whole_stack"]
    print (whole_stack.shape)

    clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
    ctr_clrs =0
    #

    area_ids = [12, 8, 13]  # left limb, left M1, left visual

    # rewarded lever pulls in seconds
    reward_times_sec = behavior_times[0]/30. + start_blue

    # get left paw, right paw and licking
    feature_ids = [2,3,7]
    behavior_times_sec = behavior_times[feature_ids]/30 + start_blue

    return reward_times_sec, behavior_times_sec

##########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'AQ2'

# 
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

allstacks = []
for session_id in sessions:
    
    try:
        reward_times_sec, behavior_times_sec = get_lever_pulls_and_behaviors(session_id, root_dir, animal_id)
    except:
        continue

    print (reward_times_sec)
    print (behavior_times_sec[0].shape)

    # make behavivor boolean vectors
    bin_width = 0.250 # bin width in seconds
    behavior_binned = np.zeros((int(10*1//bin_width)))
    print (behavior_binned.shape)
    names = ['left paw','right paw','licking']
    # loop over lever pulls:

    behaviors = [0,1,2]
    stacks = []
    for behavior in behaviors: 
        vector_stack = []
        for k in range(reward_times_sec.shape[0]):
            temp1 = reward_times_sec[k]
            temp_behavior = behavior_binned.copy()
            for p in range(behavior_times_sec[behavior].shape[0]):
                temp2 = behavior_times_sec[behavior][p]
                diff = temp1-temp2
                if diff<=10 and diff>0:
                    # find bin where it belongs
                    idx = int(behavior_binned.shape[0] - diff/bin_width)
                    #print (diff, idx)
                    temp_behavior[idx] =1

            vector_stack.append(temp_behavior)

        vector_stack = np.array(vector_stack)
        print (vector_stack.shape)

        #
        if False:
            clrs1 = ['Greys','Purples','copper']
            plt.figure(figsize=(10,5))
            clb = plt.imshow(vector_stack,
                             aspect='auto',
                             extent=[-10,0,0,vector_stack.shape[0]],
                             cmap=clrs1[behavior]
                            )
            plt.colorbar()
            plt.xlabel("Time (sec)")
            plt.ylabel("Trial #")

            plt.savefig('/home/cat/'+str(behavior)+'.svg')
            plt.close()

        stacks.append(vector_stack)
        
    allstacks.append(stacks)
    #plt.show()

# sessions:  110
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(9, 1800, 16)
(9,)
Shifts:  [-5.434  2.633 -3.467]
start blue:  2.6
(128, 128, 16)
(16, 40000)
[  32.7         338.13333333  346.76666667  360.23333333  447.13333333
  513.26666667  990.6        1160.13333333 1254.96666667 1278.16666667]
(1587,)
(40,)
(10, 40)
(10, 40)
(10, 40)
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(27, 1800, 16)
(9,)
Shifts:  [4.913 5.326 5.063]
start blue:  2.2666666666666666
(128, 128, 16)
(16, 40000)
[  41.06666667  118.06666667  123.23333333  223.9         381.5
  506.16666667  528.63333333  546.96666667  579.16666667  595.43333333
  634.53333333  671.1         687.7         719.46666667  733.1
  746.5         798.2         988.36666667 1004.43333333 1031.66666667
 1126.36666667 1130.8        1137.3        1143.43333333 1151.76666667

In [4]:
################################
############ RUN PCA ###########
################################
print (len(allstacks), len(allstacks[0]), allstacks[0][0].shape)

clrs2 = ['black','purple','khaki']
from sklearn.decomposition import PCA
a1 = []
for p in range(3):
    
    #
    temp_array =[]
    for k in range(len(allstacks)):
        temp2 = allstacks[k][p]
        temp_array.append(temp2)
    temp_array = np.vstack(temp_array)
        
    X = temp_array
    # run pca
    pca = PCA(n_components=2)
    print ("input to pca: ", X.shape)
    X_pca = pca.fit_transform(X)
    print (" pca done: ", X_pca.shape)

    fig=plt.figure(figsize=(8,8))
    plt.scatter(X_pca[:,0],
                X_pca[:,1],
                c=clrs2[p],
                #cmap = clrs1[p],
                s=100,
               # alpha=.8
               )
    plt.savefig('/home/cat/'+str(p)+'.svg')
    plt.close()
    #plt.show()
    a1.append(X)
    
# stack all feature vecotrs
X = np.vstack(np.array(a1).transpose(0,2,1)).T
print (X.shape)

# run pca
pca = PCA(n_components=2)
print ("input to pca: ", X.shape)
X_pca = pca.fit_transform(X)
print (" pca done: ", X_pca.shape)

fig=plt.figure(figsize=(8,8))
plt.scatter(X_pca[:,0],
            X_pca[:,1],
            c='blue',
            edgecolor='black',
            #cmap = clrs1[p],
            s=50,
           # alpha=.8
           )
if True:
    plt.savefig('/home/cat/sequences_AQ2.svg')
    plt.close()
else:
    plt.show()



40 3 (10, 40)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
input to pca:  (2726, 40)
 pca done:  (2726, 2)
(2726, 120)
input to pca:  (2726, 120)
 pca done:  (2726, 2)
